# Модель сегментации объектов на изображении

## Содержание
* [Постановка задачи](#Постановка-задачи)
* [Датасет](#Датасет)
* [Архитектура](#Архитектура)
* [Выбор гиперпараметров](#Выбор-гиперпараметров)
* [Обучение](#Обучение)
* [Инференс](#Инференс)
* [Оценка модели](#Оценка-модели)
* [Скорость работы](#Скорость-работы)
* [Критика](#Критика)
* [Краткое резюме](#Краткое-резюме)
* [Предложения по улучшению и развитию модели](#Предложения-по-улучшению-и-развитию-модели)

## Постановка задачи

## Датасет

In [ ]:
import sys
sys.path.append('../')

import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import batchflow as bf

from batchflow import F,V,B,C
from batchflow import FilesIndex, Dataset
from batchflow.models.tf import UNet
from batchflow.research import Research, Results, Option

from aerial_batch import AerialBatch, get_origs

%env CUDA_VISIBLE_DEVICES=0

In [ ]:
ind = FilesIndex(path='../ignore/deepglobe/train/*.jpg')
ads = Dataset(ind, AerialBatch)
ads.split(0.8, shuffle=False)

In [ ]:
def make_mask(mask):
    """
    """
    mask[mask > 2] = 0    
    return mask

## Архитектура

Используется модель UNet, реализованная в библиотеке ```batchflow.models.tf```.

Общее описание архитектуры модели доступно по ссылке [https://analysiscenter.github.io/batchflow/api/batchflow.models.tf.unet.html](https://analysiscenter.github.io/batchflow/api/batchflow.models.tf.unet.html)

Выбранная конфигурация модели:

Задание конфигурации модели:

In [ ]:
model_config = {
    'initial_block/inputs': 'images',
    'inputs': dict(images={'shape': (448,448,3)}, 
                   masks={'name':'targets',
                          'shape': (448,448,1)}),
    'filters': [32, 64, 128, 256],
    'head/num_classes': 3,
    'loss':'ce',
    'optimizer': 'Adam',
    'output': ['proba']
}

## Обучение

Обучение проиcходит итерациями по батчам. Итерации состоит из:
* генерации очередной порции данных
* обновления весов нейронной сети.

Для обучения выбран оптимизатор Adam со стандартными параметрами:
* ```learning_rate``` = 0.001
* ```beta1``` = 0.9
* ```beta2``` = 0.999
* ```epsilon``` = 1e-08.

Функция потерь - cross-entropy.

Схема обхода датасета:
* ```batch_size``` = 20
* количество итераций = 300.

In [ ]:
batch_size = 1
n_epochs = 5

Описание пайплайна обучения:

In [ ]:
crop_shape = (448, 448)

train_template = (
    bf.Pipeline()
      .load(ind, fmt='image', components='images')
      .load(ind, fmt='mask', components='masks')
#     .augmentations!
#       .resize(size=(1000, 1000), src=['images'], dst=['images'])
#       .resize(size=(1000, 1000), src=['masks'], dst=['masks'])
      .apply_transform(get_origs, crop_shape, p=1., src='masks', dst='origs')
      .crop(shape=crop_shape, src=['images', 'origs'], dst='images')
      .crop(shape=crop_shape, src=['masks', 'origs'], dst='masks')
      .to_array(src='images', dst='images')
      .to_array(src='masks', dst='masks')
      .apply_transform(make_mask, src='masks', dst='masks')
      .init_variable('loss', init_on_each_run=list)
      .init_model('dynamic', UNet, 'unet', model_config)
      .train_model('unet', images=B('images'), targets=B('masks'),
                      fetches='loss', save_to=V('loss'), mode='a')
).run(batch_size, shuffle=True, lazy=True)

Запуск обучения  с размером батча 2:

In [ ]:
train_pipeline = (train_template << ads.train)

In [ ]:
train_pipeline.run(batch_size, n_epochs=n_epochs, bar=True)

График функции потерь в зависимости от числа итераций:

In [ ]:
plt.plot(train_pipeline.get_variable('loss'))

## Оценка модели

Качество модели определяется средним значением метрики IOU (intersection over union) по выборке из 1000 картинок.

Описание процедуры расчета метрики:

In [ ]:
test_template = (
    bf.Pipeline()
      .load(ind, fmt='image', components='images')
      .load(ind, fmt='mask', components='masks')
#     .augmentations!
#       .resize(size=(1000, 1000), src=['images'], dst=['images'])
#       .resize(size=(1000, 1000), src=['masks'], dst=['masks'])
      .apply_transform(get_origs, crop_shape, src='masks', dst='origs')
      .crop(shape=crop_shape, src=['images', 'origs'], dst='images')
      .crop(shape=crop_shape, src=['masks', 'origs'], dst='masks')
      .to_array(src='images', dst='images')
      .to_array(src='masks', dst='masks')
      .init_variable('batch_predictions', init_on_each_run=list)
      .init_variable('predictions', init_on_each_run=list)
      .init_variable('metrics', init_on_each_run=None)
      .init_variable('images', init_on_each_run=list)
      .init_variable('masks', init_on_each_run=list)
      .import_model('unet', train_pipeline)
      .predict_model('unet', images=B('images'), targets=B('masks'),
                   fetches=['proba'], save_to=[V('batch_predictions')], mode='w')
      .gather_metrics('segmentation', targets=B('masks'), predictions=V('batch_predictions'),
                      fmt='proba', save_to=V('metrics'), mode='u')
      .update_variable('images', B('images'), mode='e')
      .update_variable('masks', B('masks'), mode='e')
      .update_variable('predictions', V('batch_predictions'), mode='e')
).run(batch_size, n_epochs=1, lazy=True)

Запуск расчета метрики

In [ ]:
test_pipeline = ((test_template << ads.test).run(batch_size, shuffle=False, 
                                                 n_epochs=1, drop_last=False, bar=True))

In [ ]:
metrics = test_pipeline.get_variable('metrics')
print("IOU: %.3f" % metrics.evaluate('iou', agg='mean'))
print("Confusion matrix:\n {}".format(metrics._confusion_matrix.sum(axis=0)))

Пример исходного изображения и предсказанной сегментации

In [ ]:
y_pred = test_pipeline.get_variable('predictions')
images = test_pipeline.get_variable('images')
masks = test_pipeline.get_variable('masks')

In [ ]:
i = np.random.randint(len(masks))

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))
ax1.imshow(images[i])
ax1.set_title('Image')
ax2.imshow(y_pred[i])
ax2.set_title('Pred')
ax3.imshow(masks[i][:,:,0])
ax3.set_title('Mask')

In [ ]:
plt.imshow(y_pred[i][:,:,1])

In [ ]:
y_pred[i][:,:,0]